In [7]:
import timesfm
import pandas as pd

In [8]:
# Loading the timesfm-2.0 checkpoint:
# For Torch
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 70611.18it/s]


In [3]:
# Loading the timesfm-1.0 checkpoint:
# For Torch
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
  )

Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 45590.26it/s]


In [9]:
pd.read_excel('/home1/gkrtod35/file/solar_seoul.xlsx')

,date,time,solar generation
0,2013-01-01,0,0
1,2013-01-01,1,0
2,2013-01-01,2,0
3,2013-01-01,3,0
4,2013-01-01,4,0
...,...,...,...
95659,2023-11-30,19,0.10472
95660,2023-11-30,20,0.10744
95661,2023-11-30,21,0.10752
95662,2023-11-30,22,0.037


In [22]:
df = pd.read_excel('/home1/gkrtod35/file/solar_seoul.xlsx')
df['date'] = pd.to_datetime(df['date'])
df['datetime'] = df['date'] + pd.to_timedelta(df['time'], unit = 'h')
df

,date,time,solar generation,datetime
0,2013-01-01,0,0,2013-01-01 00:00:00
1,2013-01-01,1,0,2013-01-01 01:00:00
2,2013-01-01,2,0,2013-01-01 02:00:00
3,2013-01-01,3,0,2013-01-01 03:00:00
4,2013-01-01,4,0,2013-01-01 04:00:00
...,...,...,...,...
95659,2023-11-30,19,0.10472,2023-11-30 19:00:00
95660,2023-11-30,20,0.10744,2023-11-30 20:00:00
95661,2023-11-30,21,0.10752,2023-11-30 21:00:00
95662,2023-11-30,22,0.037,2023-11-30 22:00:00


In [25]:
df[['datetime','solar generation']]

,datetime,solar generation
0,2013-01-01 00:00:00,0
1,2013-01-01 01:00:00,0
2,2013-01-01 02:00:00,0
3,2013-01-01 03:00:00,0
4,2013-01-01 04:00:00,0
...,...,...
95659,2023-11-30 19:00:00,0.10472
95660,2023-11-30 20:00:00,0.10744
95661,2023-11-30 21:00:00,0.10752
95662,2023-11-30 22:00:00,0.037


In [27]:
# 컬럼 이름 변경 및 unique_id 컬럼 추가
df_renamed = df[['datetime', 'solar generation']].rename(columns={'datetime':'ds'})
df_renamed['unique_id'] = 'solar_gen'
df_renamed

,ds,solar generation,unique_id
0,2013-01-01 00:00:00,0,solar_gen
1,2013-01-01 01:00:00,0,solar_gen
2,2013-01-01 02:00:00,0,solar_gen
3,2013-01-01 03:00:00,0,solar_gen
4,2013-01-01 04:00:00,0,solar_gen
...,...,...,...
95659,2023-11-30 19:00:00,0.10472,solar_gen
95660,2023-11-30 20:00:00,0.10744,solar_gen
95661,2023-11-30 21:00:00,0.10752,solar_gen
95662,2023-11-30 22:00:00,0.037,solar_gen


In [29]:
forecast_df = tfm.forecast_on_df(
    inputs=df_renamed,
    freq="h",  # hourly 데이터임을 명시
    value_name="solar generation",
    num_jobs=-1
)
forecast_df

Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,solar_gen,2023-12-01 00:00:00,-0.171914,-0.114405,-0.089622,-0.095196,-0.151745,-0.171914,-0.176630,-0.172796,-0.144502,-0.125860
1,solar_gen,2023-12-01 01:00:00,-0.179737,-0.205112,-0.158948,-0.169206,-0.192400,-0.179737,-0.173212,-0.173935,-0.129344,-0.099275
2,solar_gen,2023-12-01 02:00:00,-0.273093,-0.286663,-0.148836,-0.226627,-0.236510,-0.273093,-0.285588,-0.290593,-0.258650,-0.243794
3,solar_gen,2023-12-01 03:00:00,-0.070120,-0.191553,-0.061893,-0.052498,-0.057794,-0.070120,-0.051124,-0.023301,0.012551,0.103706
4,solar_gen,2023-12-01 04:00:00,0.006795,-0.094627,0.005810,0.017413,0.010247,0.006795,0.049396,0.054744,0.107398,0.275232
...,...,...,...,...,...,...,...,...,...,...,...,...
123,solar_gen,2023-12-06 03:00:00,0.177609,-0.243250,-0.004844,0.077595,0.123835,0.177609,0.235325,0.307399,0.447376,0.589691
124,solar_gen,2023-12-06 04:00:00,0.092396,-0.337025,-0.069012,-0.005983,0.015345,0.092396,0.151694,0.201493,0.316956,0.521871
125,solar_gen,2023-12-06 05:00:00,0.083052,-0.471114,-0.194117,-0.093285,-0.013545,0.083052,0.141956,0.233284,0.370754,0.636460
126,solar_gen,2023-12-06 06:00:00,0.197931,-0.449180,-0.137729,-0.003306,0.092190,0.197931,0.337521,0.452458,0.614131,0.938499


TypeError: dtype datetime64[ns] cannot be converted to timedelta64[ns]